In [1]:
import pandas as pd
df=pd.read_csv('Dataset.csv',encoding='latin-1')
df.head()

,ï»¿label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
df['label']=df[df.columns[0]]
df=df[['text','label']]
df.head()

,text,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [3]:
import string
punctuations=string.punctuation
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lem=WordNetLemmatizer()
stopword_list=stopwords.words('english')
def _clean_text(raw_text):
    
    clean_text=raw_text.lower()
    clean_text="".join(c for c in clean_text if c not in punctuations)
    words=clean_text.split()
    words=[w for w in words if w not in stopword_list]
    words=[lem.lemmatize(word,'v') for word in words]
    words=[lem.lemmatize(word,'n') for word in words]
    clean_text=' '.join(words)
    
    
    return clean_text

In [4]:
df['cleaned']=df['text'].apply(_clean_text)

In [5]:
df['word_count']=df['text'].apply(lambda x: len(x.split()))
df['word_count_clean']=df['cleaned'].apply(lambda x: len(x.split()))


df['char_count']=df['text'].apply(lambda x: len(x.replace(" ",'')))
df['char_count_clean']=df['cleaned'].apply(lambda x: len(x.replace(" ","")))


df['num']=df['cleaned'].apply(lambda x: sum([1 if w.isdigit() else 0 for w in x.split()]))

In [6]:
df.head()

,text,label,cleaned,word_count,word_count_clean,char_count,char_count_clean,num
0,"Go until jurong point, crazy.. Available only ...",ham,go jurong point crazy available bugis n great ...,20,16,92,67,0
1,Ok lar... Joking wif u oni...,ham,ok lar joke wif u oni,6,6,24,16,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry 2 wkly comp win fa cup final tkts 2...,28,23,128,113,3
3,U dun say so early hor... U c already then say...,ham,u dun say early hor u c already say,11,9,39,27,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,nah dont think go usf live around though,13,8,49,33,0


In [7]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cvz=CountVectorizer()
cvz.fit(df['cleaned'].values)

count_vect=cvz.transform(df['cleaned'].values)

In [8]:
count_vect

<5572x8206 sparse matrix of type '<class 'numpy.int64'>'
	with 46827 stored elements in Compressed Sparse Row format>

In [9]:
word_tf=TfidfVectorizer(max_features=500)
word_tf.fit(df['cleaned'].values)

word_vec_tfidf=word_tf.transform(df['cleaned'].values)

In [10]:
word_vec_tfidf

<5572x500 sparse matrix of type '<class 'numpy.float64'>'
	with 28313 stored elements in Compressed Sparse Row format>

In [11]:
tfidf=dict(zip(word_tf.get_feature_names(),word_tf.idf_))
tfidf_df=pd.DataFrame().from_dict(tfidf,orient='index')
tfidf_df.columns=['word_tfidf']
tfidf_df.head()

,word_tfidf
10,6.581166
100,5.936809
1000,5.754488
150,5.888019
150p,6.406813


In [12]:
from scipy.sparse import hstack,csr_matrix

meta_features=['word_count', 'word_count_clean',
       'char_count', 'char_count_clean', 'num']

feature_set1=df[meta_features]

train=hstack([word_vec_tfidf,csr_matrix(feature_set1)],'csr')
train

<5572x505 sparse matrix of type '<class 'numpy.float64'>'
	with 51813 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.preprocessing import LabelEncoder
target=df['label'].values
target=LabelEncoder().fit_transform(target)
target

array([0, 0, 1, ..., 0, 0, 0])

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train,target)

In [15]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(4179, 505) (4179,)
(1393, 505) (1393,)


In [16]:
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import ensemble
from sklearn.metrics import accuracy_score

In [17]:
model=naive_bayes.MultinomialNB()
model.fit(X_train,y_train)
pred=model.predict(X_test)
print(accuracy_score(pred,y_test))

0.9763101220387652


In [19]:
model=LogisticRegression()
model.fit(X_train,y_train)
pred=model.predict(X_test)
print(accuracy_score(pred,y_test))

0.9748743718592965


/Users/guptadhruv51/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [20]:
model=svm.SVC()
model.fit(X_train,y_train)
pred=model.predict(X_test)
print(accuracy_score(pred,y_test))

0.9461593682699211


In [22]:
model=ensemble.ExtraTreesClassifier()
model.fit(X_train,y_train)
pred=model.predict(X_test)
print(accuracy_score(pred,y_test))

0.9770279971284996
